In [35]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from datetime import datetime

#pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

%matplotlib inline
import matplotlib.pyplot as plt

# 0. Numpy vs Torch

In [34]:
start = datetime.now()

np.random.seed(0)

N, D = 3, 4

x = np.random.randn(N, D)
y = np.random.randn(N, D)
z = np.random.randn(N, D)

a = x + y
b = a + z
c = np.sum(b)

grad_c = 1.0
grad_b = grad_c * np.ones((N, D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_y = grad_a * y
grad_x = grad_a * x

print(grad_x)
print(grad_y)
print(grad_z)
print(datetime.now()-start)

[[ 1.76405235  0.40015721  0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788  0.95008842 -0.15135721]
 [-0.10321885  0.4105985   0.14404357  1.45427351]]
[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
0:00:00.001001


In [56]:
start = datetime.now()

N, D = 3, 4

x = Variable(torch.randn(N, D).cuda(), requires_grad=True)
y = Variable(torch.randn(N, D).cuda(), requires_grad=True)
z = Variable(torch.randn(N, D).cuda(), requires_grad=True)

a = x + y
b = a + z
c = torch.sum(b)

c.backward(gradient=torch.cuda.FloatTensor([1.0]))

print(x.grad)
print(y.grad)
print(z.grad)
print(datetime.now()-start)

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([1]) and output[0] has a shape of torch.Size([]).

# 1. Create Tensor

## 1) random numbers

In [2]:
# torch.rand(sizes) -> (0, 1)

x = torch.rand(2, 3)
x

tensor([[0.7612, 0.1151, 0.6636],
        [0.0995, 0.6414, 0.8569]])

In [3]:
# torch.randn(sizes) -> Z(0, 1)
x = torch.randn(2, 3)
x

tensor([[ 1.4508, -0.7537,  0.6018],
        [ 1.0055,  1.1899, -0.9193]])

In [4]:
# torch.randperm(n) -> permutation of 0-n
x = torch.randperm(5)
x

tensor([1, 2, 0, 3, 4])

## 2) zeros, ones, arange

In [5]:
# torch.zeros(2, 3) -> [[0, 0, 0], [0, 0, 0]]
x = torch.zeros(2, 3)
x

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [6]:
# torch.ones(2, 3) -> [[1. 1. 1]. [1, 1, 1]]
x = torch.ones(2, 3)
x

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [7]:
# torch.arange(start, end , step=1) -> [start, end) with step
x = torch.arange(0, 3, step = 0.5)
x

tensor([0.0000, 0.5000, 1.0000, 1.5000, 2.0000, 2.5000])

## 3) Tensor Data Type

In [8]:
# torch.FloatTensor(size or list)
x = torch.FloatTensor(2, 3)
x

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [9]:
# torch.FloatTensor(size or list)
x = torch.FloatTensor([2, 3])
x

tensor([2., 3.])

In [10]:
# tensor.type_as(tensor_type)
x = x.type_as(torch.IntTensor())
x

tensor([2, 3], dtype=torch.int32)

## 4) Numpy to Tensor, Tensor to Numpy

In [11]:
# torch.from_numpy(ndarray) -> Tensor

x1 = np.ndarray(shape=(2, 3), dtype=int, buffer=np.array([1,2,3,4,5,6]))
x2 = torch.from_numpy(x1)

x2

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

In [12]:
# torch.nmumpy() -> ndarray

x3 = x2.numpy()
x3

array([[1, 2, 3],
       [4, 5, 6]])

## 5) Tensor on CPU & GPU

In [13]:
x = torch.FloatTensor([[1,2,3], [4,5,6]])
x

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [14]:
x_gpu = x.cuda()
x_gpu

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')

In [15]:
x_cpu = x_gpu.cpu()
x_cpu

tensor([[1., 2., 3.],
        [4., 5., 6.]])

## 6) Tensor Size

In [16]:
# tensor.size() -> indexing also possible

x = torch.FloatTensor(10, 12, 3, 3)

x.size()

torch.Size([10, 12, 3, 3])

# 2. Indexing, Slicing, Joining

## 1) Indexing

In [17]:
# torch.index_select(input, dim, index)

x = torch.rand(4, 3)
out = torch.index_select(x, # input tensor
                         0, # dim. 0==row, 1==column, 2....
                         torch.LongTensor([0, 3])) # index 여기서는 0행과, 3행

x,out

(tensor([[0.9214, 0.3935, 0.6402],
         [0.4025, 0.8451, 0.6416],
         [0.0895, 0.8513, 0.6680],
         [0.9623, 0.1609, 0.1328]]),
 tensor([[0.9214, 0.3935, 0.6402],
         [0.9623, 0.1609, 0.1328]]))

In [18]:
# pythonic indexing also works

x[:, 0], x[0, :], x[0:2, 0:2]

(tensor([0.9214, 0.4025, 0.0895, 0.9623]),
 tensor([0.9214, 0.3935, 0.6402]),
 tensor([[0.9214, 0.3935],
         [0.4025, 0.8451]]))

In [19]:
# torch.masked_select(input, mask)
# pooling과 비슷하네요. 텐서에서 해당 좌표 값만 추출합니다.

x = torch.randn(2, 3)
mask = torch.ByteTensor([[0, 0, 1], [0, 1, 0]])
out = torch.masked_select(x, mask)

x, mask, out

C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:6: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:733.)
  


(tensor([[-0.0317,  0.0618,  0.5288],
         [-0.1193,  0.3570,  0.7282]]),
 tensor([[0, 0, 1],
         [0, 1, 0]], dtype=torch.uint8),
 tensor([0.5288, 0.3570]))

## 2) Joining

In [20]:
# torch.cat(seq, dim=0) -> concatenate tensor along dim

x = torch.FloatTensor([[1, 2, 3], [4, 5, 6]])
y = torch.FloatTensor([[-1, -2, -3], [-4, -5, -6]])
z1 = torch.cat([x, y], dim=0) # 0은 row니까 밑으로 붙입니다.
z2 = torch.cat([x, y], dim=1) # 1은 col니까 옆으로 붙입니다.

x, y, z1, z2

(tensor([[1., 2., 3.],
         [4., 5., 6.]]),
 tensor([[-1., -2., -3.],
         [-4., -5., -6.]]),
 tensor([[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [-1., -2., -3.],
         [-4., -5., -6.]]),
 tensor([[ 1.,  2.,  3., -1., -2., -3.],
         [ 4.,  5.,  6., -4., -5., -6.]]))

In [21]:
# torch.stack(sequence, dim=0) -> stack along new dim

x = torch.FloatTensor([[1, 2, 3], [4, 5, 6]])
x_stack = torch.stack([x, x, x, x], dim=0)

x_stack

tensor([[[1., 2., 3.],
         [4., 5., 6.]],

        [[1., 2., 3.],
         [4., 5., 6.]],

        [[1., 2., 3.],
         [4., 5., 6.]],

        [[1., 2., 3.],
         [4., 5., 6.]]])

## 3) Slicing

In [22]:
# torch.chunk(tensor, chunks, dim=0) -> tensor into num chunks

x_1, x_2 = torch.chunk(z1, 2, dim=0)
y_1, y_2, y_3 = torch.chunk(z1, 3, dim=1)

z1, x_1, x_2, y_1, y_2, y_3

(tensor([[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [-1., -2., -3.],
         [-4., -5., -6.]]),
 tensor([[1., 2., 3.],
         [4., 5., 6.]]),
 tensor([[-1., -2., -3.],
         [-4., -5., -6.]]),
 tensor([[ 1.],
         [ 4.],
         [-1.],
         [-4.]]),
 tensor([[ 2.],
         [ 5.],
         [-2.],
         [-5.]]),
 tensor([[ 3.],
         [ 6.],
         [-3.],
         [-6.]]))

In [23]:
# torch.split(tensor, split_size, dim=0) -> split into specific size

x1, x2 = torch.split(z1, 2, dim=0)
y1 = torch.split(z1, 2, dim=1)

z1, x1, x2, y1

(tensor([[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [-1., -2., -3.],
         [-4., -5., -6.]]),
 tensor([[1., 2., 3.],
         [4., 5., 6.]]),
 tensor([[-1., -2., -3.],
         [-4., -5., -6.]]),
 (tensor([[ 1.,  2.],
          [ 4.,  5.],
          [-1., -2.],
          [-4., -5.]]),
  tensor([[ 3.],
          [ 6.],
          [-3.],
          [-6.]])))

## 4) squeezing

In [24]:
# torch.squeeze(input, dim=None) -> reduce dim by 1

x1 = torch.FloatTensor(10, 1, 3, 1, 4)
x2 = torch.squeeze(x1)

x1.size(), x2.size()

(torch.Size([10, 1, 3, 1, 4]), torch.Size([10, 3, 4]))

In [25]:
# torch.unsueeze(input, dim=None) -> add dim by 1

x1 = torch.FloatTensor(10, 3, 4)
x2 = torch.unsqueeze(x1, dim=0)

x1.size(), x2.size()

(torch.Size([10, 3, 4]), torch.Size([1, 10, 3, 4]))

# 3. Initialization

In [29]:
import torch.nn.init as init

In [32]:
# 텐서가 랜덤값이 아닌 특정 분포를 지정해주고 싶을 때 씁니다.

x1 = init.uniform(torch.FloatTensor(3, 4), a=0, b=9) # 0~9로 이니셜라이징 해주세요.
x2 = init.normal(torch.FloatTensor(3, 4), std=0.2)
x3 = init.constant(torch.FloatTensor(3, 4), 3.1415)

x1, x2, x3

C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  """Entry point for launching an IPython kernel.
C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:2: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  
C:\Users\Wyatt\anaconda3\envs\wyatt_torch\lib\site-packages\ipykernel_launcher.py:3: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  This is separate from the ipykernel package so we can avoid doing imports until


(tensor([[1.5830, 6.3767, 3.6515, 1.3709],
         [0.6528, 4.7532, 4.7284, 0.9978],
         [6.8952, 6.4114, 1.3065, 7.1485]]),
 tensor([[ 0.2944, -0.0201,  0.0679,  0.0289],
         [-0.2082, -0.1771,  0.3685, -0.0480],
         [-0.2725,  0.5291,  0.0116, -0.1260]]),
 tensor([[3.1415, 3.1415, 3.1415, 3.1415],
         [3.1415, 3.1415, 3.1415, 3.1415],
         [3.1415, 3.1415, 3.1415, 3.1415]]))